# Dynamics of Politicians

The model that we discussed in this post and the previous one is a pretty basic dynamic programming problem. In the context of econ, the framework of being elected vs. not elected is pretty similar to the job search question. Just as a brief recap, the idea of dynamic programming is to take a difficult multi-step problem and break them down into sequence of similar decision steps over time. 

In the politician problem that we have been discussing, the key is to identify how to break down this complicated issue of what decisions a politican makes

In [ ]:
# Load in some helpful packages
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Initialize the state space